<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class RotationPredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(RotationPredictionModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

def train_self_supervised(model, dataloader, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        for data, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Generate synthetic data for example usage
input_dim = 64
num_classes = 4
batch_size = 16

data = torch.randn(100, input_dim)  # 100 samples with input dimension 64
labels = torch.randint(0, num_classes, (100,))  # 100 labels with values between 0 and 3 (4 classes)

dataset = TensorDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage
model = RotationPredictionModel(input_dim=input_dim, num_classes=num_classes)
train_self_supervised(model, dataloader, epochs=10)